<a href="https://colab.research.google.com/github/urbanows/ipynb/blob/master/GraylogDynamoDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract aggregated date from GaryLog an populated DynamoDB 


DynamoDB API Guide: http://boto3.readthedocs.io/en/latest/guide/dynamodb.html

boto3 documentation: https://boto3.readthedocs.io/en/latest/guide/configuration.html 

Install pysb

In [ ]:
%%bash

pip install sciencebasepy

Install boto3 for Amazon DynamoDB

In [ ]:
%%bash
pip install boto3

In [ ]:
#from pysb import SbSession
import time
import os
import sciencebasepy
from getpass import getpass
import bs4
import requests
import base64
import subprocess
#import boto3
import datetime
import getpass
import random
import string

Establish DynamoDB connection

In [ ]:
# Do not hard code credentials
session = boto3.session(
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    aws_session_token=SESSION_TOKEN,
)

# Get the service resource.
dynamodb = session.resource('dynamodb')

1) Pull items for IGSN assignment out of ScienceBase

In [ ]:
#identify targeted collection
# loginc prompts for a password
sb = sciencebasepy.SbSession("beta").loginc("surbanowski@usgs.gov")
#sb = sciencebasepy.SbSession("beta")

itemID = '57b5fab8e4b0062befcc4ad7' 
sbItem=sb.get_item(itemID)
itemUrl=sbItem['link']['url']
print(itemUrl)


··········
https://beta.sciencebase.gov/catalog/item/57b5fab8e4b0062befcc4ad7


In [ ]:

#Rev v2 will support multile items (children)
#List = sb.get_child_ids(itemID)
#print List

Build next identifier

Assign IGSN prefix (USWA (for Washington GS), example IGSN: USWA1234)

In [ ]:
#Test prefix is 20.500.11812
#Prod prefix is 20.500.10273
IGSN_HANDLE="20.500.11812/"
#Dynamic Determined by org (e.g. "USUT")
IGSN_PREFIX="USGS-"

#TODO: Build key according to: http://igsn.github.io/syntax/
#Gen random key.  seems like a good compromise between key length and readability
#Look at removing zero and upper_case 'o'
igsnKey=''.join([random.choice(string.ascii_letters+string.digits+'-_') for ch in range(8)])

#test prefix could be used: 20.500.11812. 
#I think this is to be used without the  test parameter.
IGSN = IGSN_HANDLE + IGSN_PREFIX + igsnKey
print(IGSN)


20.500.11812/USGS-iMuqDgom


DynamoDB: Lookup ID to ensure no duplication with existing ids

In [ ]:
#TODO: Should be case insensitive
table = dynamodb.Table('igsnRegistration')
response = table.get_item(
    Key={
        'igsnId': IGSN
    }
)

#If size > 0, try again
if 0<len(response):
  print("Error: IGSN not unique")


Login to IGSN

In [ ]:
#need to build from user input
user=input()
passwd = getpass.getpass()
authString=user+":"+passwd

authStrEncoded = base64.standard_b64encode(authString.encode('utf-8'))

USGS.USGS
··········


 5) Register IGSN with IGSN Metadata Store.  API Reference at:
 https://doidb.wdc-terra.org/igsn/static/apidoc
 
 Call IGSN web service to register IGSN. In test mode for prototype work (?testMode=true)

1) Create IGSN: POST request:

igsn={igsn}
url={http://www.sciencebase.gov/catalog/item/5989cdece4b09fa1cb0c54b3}

(ISGN is created with assigned URL)

In [ ]:
URL_TEST_FLAG="?testMode=true"
MINT_URL = "https://doidb.wdc-terra.org/igsn/igsn"+URL_TEST_FLAG

MINT_PARAMS = "igsn:"+IGSN+"\nurl:"+itemUrl
print(MINT_PARAMS)
MINT_HEADERS = {'Content-Type':'text/plain;charset=UTF-8',
                'Authorization':'Basic ' + authStrEncoded.decode('utf-8')}

# sending get request and saving the response as response object
r = requests.post(url = MINT_URL, data = MINT_PARAMS, headers=MINT_HEADERS)

#Check for 201 response and HANDLE_ALREADY_EXISTS
print(r.content)

VVNHUy5VU0dTOk5HR0RQUDEyMyE=
igsn:20.500.11812/USGS-iMuqDgom
url:https://beta.sciencebase.gov/catalog/item/57b5fab8e4b0062befcc4ad7
b'OK (only test mode!)'


2) Provide metadata for new IGSN

In [ ]:
#TODO: Org name only?
registrantName="Urbanowski"
#TODO: Determine if OrcID needed
orcId="0000-0001-5030-6719"
currDateTime=datetime.datetime.now().isoformat()

XML='<sample xmlns="http://igsn.org/schema/kernel-v.1.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://igsn.org/schema/kernel-v.1.0 http://doidb.wdc-terra.org/igsn/schemas/igsn.org/schema/1.0/igsn.xsd"><sampleNumber identifierType="igsn">'+IGSN+'</sampleNumber><registrant><registrantName>'+registrantName+'</registrantName><nameIdentifier nameIdentifierScheme="orcid">'+orcId+'</nameIdentifier></registrant><log><logElement event="registered" timeStamp="'+currDateTime+'"/><logElement event="submitted" timeStamp="'+currDateTime+'"/></log></sample>'
METADATA_BASE_URL="https://doidb.wdc-terra.org/igsn/metadata/"

METADATA_HEADERS = {'Content-Type':'text/xml;charset=UTF-8',
                   'Authorization':'Basic ' + authStrEncoded.decode('utf-8')}

# sending get request and saving the response as response object
r = requests.post(url = METADATA_BASE_URL+IGSN+URL_TEST_FLAG, data=XML, headers=METADATA_HEADERS)
#r = requests.post(url = METADATA_URL+IGSN+URL_TEST_FLAG, auth=HTTPBasicAuth('user', 'pass'), data=XML, headers=METADATA_HEADERS)

#Check for 201 response
print(r.status_code)

201


5) Add entry to data store with identifier and SB id/url

In [ ]:
table.put_item(
   Item={
        'igsnId': IGSN,
        'url': itemUrl,
    }
)

6) Write IGSN to ScienceBase items

Lock record?

In [ ]:
# loginc prompts for a password
sb = SbSession("beta").loginc("surbanowski@usgs.gov")

··········


In [ ]:
igsn_update = {'id':itemID, "identifiers": [ { 'type': 'IGSN', 'scheme': 'https://www.sciencebase.gov/vocab/category/item/identifier', 'key': IGSN } ]}
    
sb.update_item(igsn_update)

{'contacts': [{'active': True,
   'company': 'GS, Room:  9210, Affiliation:  EMPLOYEE',
   'contactType': 'person',
   'email': 'surbanowski@usgs.gov',
   'firstName': 'Shayne',
   'jobTitle': 'Computer Scientist',
   'lastName': 'Urbanowski',
   'middleName': 'R',
   'name': 'Shayne R Urbanowski',
   'oldPartyId': 64532,
   'orcId': '0000-0001-5030-6719',
   'organization': {'displayText': 'Scientific Data Integration and Visualization'},
   'primaryLocation': {'building': 'DFC Bldg 810',
    'buildingCode': 'KBT',
    'location': 'Denver Federal Center',
    'name': 'Shayne R Urbanowski/GIO/CONT/USGS/DOI - Primary Location',
    'officePhone': '303-202-4089',
    'roomNumber': '9210',
    'streetAddress': {'city': 'Lakewood',
     'country': 'US',
     'line1': 'W 6th Ave Kipling St',
     'state': 'CO',
     'zip': '80225'}},
   'type': 'Author'}],
 'distributionLinks': [{'files': '',
   'name': '',
   'rel': 'alternate',
   'title': 'KML Service',
   'type': 'kml',
   'typeLabel': 

Check results

In [ ]:
sb.get_item(itemID)

ConnectionError: ignored

Future step 0:

Bulk load objects (collections and samples) into ScienceBase creating parent and child items